# 🧠 NSSS Security Auditor - Colab Runner

This notebook deploys the Neuro-Symbolic Software Security (NSSS) Auditor server on Google Colab.

**Features:**
- **Hybrid Sync:** Keeps code on Drive (persistent) but runs on Colab VM (fast).
- **Model Persistence:** Links heavy model weights directly from Drive (no re-downloading).
- **Auto-Tunneling:** Exposes the server via Ngrok automatically (supports Static Domains).

**Prerequisites:**
1.  **Google Drive Folder:** `My Drive/NSSS_Project`
2.  **Ngrok Token:** Saved in `My Drive/NSSS_Project/ngrok_token.txt` (or enter manually).
3.  **Runtime:** GPU (T4 or better).

In [ ]:
#@title 1. Configuration & Drive Mount
import os
from google.colab import drive

#@markdown ### 📂 Project Settings
DRIVE_ROOT = "/content/drive/MyDrive/NSSS_Project" #@param {type:"string"}
GITHUB_REPO = "https://github.com/Hieureal1305/Neuro-Symbolic_Software_Security.git" #@param {type:"string"}

#@markdown ### 🔄 Sync Options
UPDATE_FROM_GITHUB = True #@param {type:"boolean"}
FORCE_REINSTALL = False #@param {type:"boolean"}

# Mount Drive
if not os.path.exists("/content/drive"):
    drive.mount("/content/drive")

print(f"✅ Google Drive mounted at /content/drive")

In [ ]:
#@title 2. Smart Sync (Drive <-> Colab)
import shutil
import subprocess

def run_cmd(cmd, cwd=None):
    print(f"⚡ Running: {cmd}")
    subprocess.run(cmd, shell=True, check=True, cwd=cwd)

# 1. Ensure Drive Project Folder Exists
if not os.path.exists(DRIVE_ROOT):
    print(f"📂 Creating project folder at {DRIVE_ROOT}...")
    os.makedirs(DRIVE_ROOT, exist_ok=True)
    # Initial Clone
    run_cmd(f"git clone {GITHUB_REPO} .", cwd=DRIVE_ROOT)
else:
    # 2. Optional Update
    if UPDATE_FROM_GITHUB:
        print("🔄 Updating code from GitHub...")
        # Check if it's a git repo
        if os.path.exists(os.path.join(DRIVE_ROOT, ".git")):
            run_cmd("git pull", cwd=DRIVE_ROOT)
        else:
            print("⚠️ Drive folder exists but is not a git repo. Skipping pull.")

# 3. Setup Workspace on Colab VM (Fast I/O)
WORKSPACE = "/content/app"

if os.path.exists(WORKSPACE):
    shutil.rmtree(WORKSPACE)

print("🚀 Copying code to Colab Runtime (for speed)...")
# Copy source code only (exclude heavy folders)
shutil.copytree(
    DRIVE_ROOT,
    WORKSPACE,
    ignore=shutil.ignore_patterns("outputs", "data", "venv", ".git", "__pycache__")
)

# 4. Symlink Heavy Folders (Model & Data) back to Drive
# This keeps 7GB model on Drive, but makes it accessible at /content/app/outputs
outputs_drive = os.path.join(DRIVE_ROOT, "outputs")
outputs_app = os.path.join(WORKSPACE, "outputs")

data_drive = os.path.join(DRIVE_ROOT, "data")
data_app = os.path.join(WORKSPACE, "data")

if not os.path.exists(outputs_drive):
    os.makedirs(outputs_drive)

if not os.path.exists(data_drive):
    os.makedirs(data_drive)

# Create Symlinks
os.symlink(outputs_drive, outputs_app)
os.symlink(data_drive, data_app)

print(f"🔗 Linked {outputs_drive} -> {outputs_app}")
print(f"🔗 Linked {data_drive} -> {data_app}")

# Set working directory
os.chdir(WORKSPACE)
print(f"📍 Working directory set to: {os.getcwd()}")

In [ ]:
#@title 3. Install Dependencies

print("📦 Installing Unsloth (Optimized for Colab)...")
try:
    import unsloth
except ImportError:
    !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --progress-bar off
    !pip install --no-deps "xformers<0.0.27" "trl<0.8.6" peft accelerate bitsandbytes --progress-bar off

print("📦 Installing Server Dependencies...")
!pip install fastapi uvicorn pyngrok python-multipart nest-asyncio --progress-bar off

# Install project requirements if needed
if os.path.exists("requirements.txt"):
    !pip install -r requirements.txt --progress-bar off

print("✅ Environment Ready!")

In [ ]:
#@title 4. Launch Server
import threading
import uvicorn
import nest_asyncio
from pyngrok import ngrok
from src.server.colab_server import app

#@markdown ### 🌐 Static Domain (Optional)
#@markdown If you have a free static domain from [Ngrok Dashboard](https://dashboard.ngrok.com/cloud-edge/domains), enter it here (e.g., `my-app.ngrok-free.app`).
NGROK_DOMAIN = "" #@param {type:"string"}

# 1. Get Ngrok Token
token_path = os.path.join(DRIVE_ROOT, "ngrok_token.txt")
NGROK_AUTH_TOKEN = "" 

if os.path.exists(token_path):
    with open(token_path, "r") as f:
        NGROK_AUTH_TOKEN = f.read().strip()
    print("🔑 Loaded Ngrok token from Drive.")
else:
    print(f"⚠️ Token file not found at {token_path}")
    NGROK_AUTH_TOKEN = input("Enter your Ngrok Authtoken: ")
    # Optionally save it back
    save_it = input("Save token to Drive for next time? (y/n): ")
    if save_it.lower() == 'y':
        with open(token_path, "w") as f:
            f.write(NGROK_AUTH_TOKEN)
        print("💾 Token saved to Drive.")

# 2. Configure Ngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 3. Open Tunnel
PORT = 8000
# Close existing tunnels if any
ngrok.kill()

if NGROK_DOMAIN:
    print(f"🔌 Connecting to static domain: {NGROK_DOMAIN}")
    public_url = ngrok.connect(PORT, domain=NGROK_DOMAIN).public_url
else:
    print("🔌 Connecting to random domain...")
    public_url = ngrok.connect(PORT).public_url

print("="*50)
print(f"🚀 SERVER IS LIVE AT: {public_url}")
print("="*50)

# 4. Run Server
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=PORT)